In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, FCC
import awkward
import dask_awkward
import numpy

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:
# fcc = FCC.get_schema()
# events = NanoEventsFactory.from_root(
#     '../coffea-fcc-analyses/data/wzp6_ee_mumuH_Hbb_ecm240/events_159112833.root:events',
#     schemaclass=fcc,
#     entry_start=100,
#     entry_stop=200,
#     metadata={'dataset':'ZH'},
#     delayed=False,
#     uproot_options={"filter_name": lambda x : "PARAMETERS" not in x}
# ).events()
fcc = FCC.get_schema()
events = NanoEventsFactory.from_root(
    '../coffea-fcc-analyses/data/wzp6_ee_mumuH_Hbb_ecm240/test_Winter2023.root:events',
    schemaclass=fcc,
    metadata={'dataset':'ZH'},
    delayed=False,
    uproot_options={"filter_name": lambda x : "PARAMETERS" not in x}
).events()

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/mapping/uproot.py:144: UserWarning: Skipping PARAMETERS/_intMap/_intMap.first as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretable by Uproot")
/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/mapping/uproot.py:144: UserWarning: Skipping PARAMETERS/_intMap/_intMap.second as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretable by Uproot")
/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/mapping/uproot.py:144: UserWarning: Skipping PARAMETERS/_floatMap/_floatMap.first as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretable by Uproot")
/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/mapping/uproot.py:144: UserWarning: Skipping PARAMETERS/_floatMap/_floatMap.second as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretabl

In [3]:
from particle import Particle
def from_pdgid(array, doubly_nested=False):
    """
    Return an awkward array of names of particles
    from an awkward array of PDGIDs of the particles
    """
    def get_name(pdgid):
        try:
            s = str(Particle.from_pdgid(pdgid))
        except:
            s = 'unknown'
        return s
        
    def PDG_finder(layout, **kwargs):
        if layout.is_numpy:
            return awkward.Array(
                [get_name(id) for id in layout.data]
            ).layout

    return awkward.transform(PDG_finder, array)

In [4]:
mc = awkward.zip({
    "mc": events.Particle,
    "name": from_pdgid(events.Particle.PDG)
})
mc.name

<Array [['e-', 'e+', ..., 'gamma', 'gamma'], ...] type='100 * var * string'>

## We will look at the decay of Kaon $K_S^0 \rightarrow pions $

$$ K_S^0 \rightarrow \pi^0 + \pi^0 $$
$$ K_S^0 \rightarrow \pi^+ + \pi^- $$

In [5]:
# Find Single K_S^0
K_S0_cut = mc.name == 'K(S)0'
K_S0 = mc[K_S0_cut]
single_K_S0_cut = awkward.num(K_S0, axis = 1) == 1
single_K_S0 = K_S0[single_K_S0_cut]
single_K_S0.name

<Array [['K(S)0'], ['K(S)0'], ..., ['K(S)0']] type='27 * var * string'>

## Test for Daughters

- The Kaon $K_S^0$ must have only pions as the daughters

In [6]:
# Find the daughters of Single K_S^0
daughters_of_K_S0 = single_K_S0.mc.get_daughters
names_of_daughters_of_K_S0 = from_pdgid(daughters_of_K_S0.PDG)
names_of_daughters_of_K_S0

<Array [[['pi+', 'pi-']], ..., [['pi+', ...]]] type='27 * var * var * ?string'>

In [7]:
# Are these valid daughter particles (pi+ or pi- or pi0)?
flat_names = awkward.ravel(names_of_daughters_of_K_S0)
is_pi_0 = flat_names == 'pi0'
is_pi_plus = flat_names == 'pi+'
is_pi_minus = flat_names == 'pi-'
names_valid = awkward.all(is_pi_0 | is_pi_plus | is_pi_minus)
print("Valid daughters? : ", names_valid )

# Do the daughters have valid charges (same or opposite)?
nested_bool = awkward.prod(daughters_of_K_S0.charge,axis=2) <= 0
charge_valid = awkward.all(awkward.ravel(nested_bool))
print("Valid charge of daughters? : ", charge_valid )
print("\nDaughters test is successful? : ", names_valid & charge_valid)

Valid daughters? :  True
Valid charge of daughters? :  True

Daughters test is successful? :  True


## Test for Parents

- These pion daughters, just generated, must point back to the single parent $K_S^0$

In [10]:
daughters = awkward.zip({
    "daughter": daughters_of_K_S0,
    "name": from_pdgid(daughters_of_K_S0.PDG)
})
daughters.name

<Array [[['pi+', 'pi-']], ..., [['pi+', ...]]] type='27 * var * var * ?string'>

In [24]:
p = daughters.daughter.get_parents

name_of_parents = from_pdgid(p.PDG)
name_of_parents

<Array [[[['K(S)0'], ['K(S)0']]], ..., [[...]]] type='27 * var * var * opti...'>

In [27]:
# Do the daughters have a single parent?
nested_bool = awkward.num(p, axis=3) == 1
daughters_have_single_parent = awkward.all(awkward.ravel(nested_bool))
print("Daughters have a single parent? : ", daughters_have_single_parent)

# Is that parent K_S^0 ?
nested_bool = name_of_parents == 'K(S)0'
daughters_have_K_S0_parent = awkward.all(awkward.ravel(nested_bool))
print("\nDaughters have a K(S)0 as a parent? : ", daughters_have_K_S0_parent)

print("\n\nTest for parents successful? : ", daughters_have_single_parent & daughters_have_K_S0_parent)

Daughters have a single parent? :  True

Daughters have a K(S)0 as a parent? :  True


Test for parents successful? :  True
